In [1]:
# Copyright 2020 Google Research. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# ==============================================================================
"""A simple example on how to use keras model for inference."""
import os
from absl import app
from absl import flags
from absl import logging
import numpy as np
from PIL import Image
import tensorflow as tf

In [2]:
import hparams_config
import inference
from tf2 import efficientdet_keras

In [3]:
model_name = 'efficientdet-d2'
config = hparams_config.get_efficientdet_config(model_name)

In [4]:
config.image_path = 'examples/undetected/example.jpg'
config.output_dir = 'examples/detected'
config.model_dir = '_logs'
config.debug = False
config.saved_model_dir = '_logs/models'
config.num_classes = 1
config.val_json_file = None
config.steps_per_execution = 1
config.max_instances_per_image = 200
config.strategy = 'gpus'
config.num_epochs = 300
config.batch_size = 4
#config.batch = 2
config.moving_average_decay = None

In [5]:
# pylint: disable=line-too-long
# Prepare images and checkpoints: please run these commands in shell.
im = Image.open(config.image_path)
im = im.resize((768,768), Image.ANTIALIAS)
im = np.array(im)
imgs = [im]
# Create model config.
config.is_training_bn = True
config.image_size = (768,768)
config.nms_configs.score_thresh = 0.4
config.nms_configs.max_output_size = 100
#config.override(hparams)

# Use 'mixed_float16' if running on GPUs.
policy = tf.keras.mixed_precision.Policy('float32')
tf.keras.mixed_precision.set_global_policy(policy)
tf.config.run_functions_eagerly(config.debug)

# Create and run the model.
model = efficientdet_keras.EfficientDetModel(config=config)
model.build((None, None, None, 3))
ckpt = tf.train.latest_checkpoint(config.model_dir)
model.load_weights(ckpt)
model.summary()

2022-03-09 14:32:30.522021: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-03-09 14:32:30.527934: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-03-09 14:32:30.528184: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-03-09 14:32:30.529091: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

/home/zverko/.local/lib/python3.8/site-packages/keras/engine/base_layer.py:1348: UserWarning: `layer.updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`layer.updates` will be removed in a future version. '


Model: ""
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
efficientnet-b2 (Model)      multiple                  7267314   
_________________________________________________________________
resample_p6 (ResampleFeature multiple                  39984     
_________________________________________________________________
resample_p7 (ResampleFeature multiple                  0         
_________________________________________________________________
fpn_cells (FPNCells)         multiple                  678479    
_________________________________________________________________
class_net (ClassNet)         multiple                  49737     
_________________________________________________________________
box_net (BoxNet)             multiple                  52788     
Total params: 8,088,302
Trainable params: 8,006,526
Non-trainable params: 81,776
___________________________________________________

In [6]:
class ExportModel(tf.Module):

  def __init__(self, model):
    super().__init__()
    self.model = model

  @tf.function
  def f(self, imgs):
    return self.model(imgs, training=False, post_mode='global')

imgs = tf.convert_to_tensor(imgs, dtype=tf.uint8)
export_model = ExportModel(model)
if config.saved_model_dir:
  tf.saved_model.save(
      export_model,
      config.saved_model_dir,
      signatures=export_model.f.get_concrete_function(
          tf.TensorSpec(shape=(None, None, None, 3), dtype=tf.uint8)))
  export_model = tf.saved_model.load(config.saved_model_dir)

2022-03-09 14:33:08.340879: W tensorflow/python/util/util.cc:348] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


/home/zverko/.local/lib/python3.8/site-packages/keras/engine/base_layer.py:1348: UserWarning: `layer.updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`layer.updates` will be removed in a future version. '


/home/zverko/.local/lib/python3.8/site-packages/keras/engine/base_layer.py:1348: UserWarning: `layer.updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`layer.updates` will be removed in a future version. '


/home/zverko/.local/lib/python3.8/site-packages/keras/engine/base_layer.py:1348: UserWarning: `layer.updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`layer.updates` will be removed in a future version. '
/home/zverko/.local/lib/python3.8/site-packages/keras/engine/base_layer.py:1348: UserWarning: `layer.updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`layer.updates` will be removed in a future version. '
/home/zverko/.local/lib/python3.8/site-packages/keras/engine/base_layer.py:1348: UserWarning: `layer.updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`layer.updates` will be removed in a future version. '
/home/zverko/.local/lib/python3.8/site-packages/keras/engine/base_l

/home/zverko/.local/lib/python3.8/site-packages/keras/engine/base_layer.py:1348: UserWarning: `layer.updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`layer.updates` will be removed in a future version. '


INFO:tensorflow:Assets written to: _logs/models/assets


INFO:tensorflow:Assets written to: _logs/models/assets


In [7]:
boxes, scores, classes, valid_len = export_model.f(imgs)

# Visualize results.
for i, img in enumerate(imgs):
  length = valid_len[i]
  img = inference.visualize_image(
      img,
      boxes[i].numpy()[:length],
      classes[i].numpy().astype(np.int)[:length],
      scores[i].numpy()[:length],
      label_map=config.label_map,
      min_score_thresh=config.nms_configs.score_thresh,
      max_boxes_to_draw=config.nms_configs.max_output_size)
  output_image_path = os.path.join(config.output_dir, str(i) + '.jpg')
  Image.fromarray(img).save(output_image_path)
  print('writing annotated image to %s' % output_image_path)

2022-03-09 14:34:27.940989: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)
2022-03-09 14:34:30.772583: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8204
2022-03-09 14:34:31.347973: W tensorflow/core/common_runtime/bfc_allocator.cc:272] Allocator (GPU_0_bfc) ran out of memory trying to allocate 1.73GiB with freed_by_count=0. The caller indicates that this is not a failure, but may mean that there could be performance gains if more memory were available.
2022-03-09 14:34:31.358091: W tensorflow/core/common_runtime/bfc_allocator.cc:272] Allocator (GPU_0_bfc) ran out of memory trying to allocate 1.79GiB with freed_by_count=0. The caller indicates that this is not a failure, but may mean that there could be performance gains if more memory were available.
2022-03-09 14:34:31.361555: W tensorflow/core/common_runtime/bfc_allocator.cc:272] Allocator (GPU_0_bfc) ran out of memory trying t

writing annotated image to examples/detected/0.jpg


In [8]:
scores

<tf.Tensor: shape=(1, 100), dtype=float32, numpy=
array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.]], dtype=float32)>

In [9]:
classes

<tf.Tensor: shape=(1, 100), dtype=float32, numpy=
array([[1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1.]], dtype=float32)>

In [10]:
config.num_classes

1

In [11]:
img.shape

(768, 768, 3)

In [12]:
boxes

<tf.Tensor: shape=(1, 100, 4), dtype=float32, numpy=
array([[[13.836461, 10.623576, 16.981905, 16.802576],
        [13.836461, 10.623576, 16.981905, 16.802576],
        [13.836461, 10.623576, 16.981905, 16.802576],
        [13.836461, 10.623576, 16.981905, 16.802576],
        [13.836461, 10.623576, 16.981905, 16.802576],
        [13.836461, 10.623576, 16.981905, 16.802576],
        [13.836461, 10.623576, 16.981905, 16.802576],
        [13.836461, 10.623576, 16.981905, 16.802576],
        [13.836461, 10.623576, 16.981905, 16.802576],
        [13.836461, 10.623576, 16.981905, 16.802576],
        [13.836461, 10.623576, 16.981905, 16.802576],
        [13.836461, 10.623576, 16.981905, 16.802576],
        [13.836461, 10.623576, 16.981905, 16.802576],
        [13.836461, 10.623576, 16.981905, 16.802576],
        [13.836461, 10.623576, 16.981905, 16.802576],
        [13.836461, 10.623576, 16.981905, 16.802576],
        [13.836461, 10.623576, 16.981905, 16.802576],
        [13.836461, 10.623576

In [13]:
np.array(im)

array([[[201, 206, 209],
        [202, 207, 210],
        [202, 207, 210],
        ...,
        [212, 208, 209],
        [204, 204, 205],
        [200, 200, 204]],

       [[201, 206, 209],
        [201, 206, 209],
        [202, 207, 210],
        ...,
        [212, 208, 208],
        [207, 206, 207],
        [201, 201, 205]],

       [[201, 206, 209],
        [202, 207, 210],
        [202, 207, 210],
        ...,
        [211, 208, 209],
        [206, 205, 207],
        [202, 203, 206]],

       ...,

       [[ 45,  47,  59],
        [ 57,  55,  68],
        [ 54,  54,  67],
        ...,
        [ 32,  41,  53],
        [ 29,  35,  49],
        [ 51,  53,  70]],

       [[ 51,  52,  64],
        [ 64,  65,  76],
        [ 69,  67,  80],
        ...,
        [ 38,  45,  58],
        [ 34,  35,  51],
        [ 50,  51,  69]],

       [[ 51,  51,  64],
        [ 53,  53,  65],
        [ 51,  49,  63],
        ...,
        [ 49,  52,  67],
        [ 47,  46,  62],
        [ 54,  53,  71]]